<a href="https://colab.research.google.com/github/Tekleab15/Context-Aware-Conversational-Search-System/blob/main/notebooks/data_preparation_and_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data preparation file**

In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# importing the required modules
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import re, os
import tensorflow as tf
import tensorflow_text as tf_text
from google.colab import drive

np.random.seed(42)
tf.random.set_seed(42)

# Verifying that this section run successfully!
print("Notebook setup complete: Libraries imported and seeds set.")

Notebook setup complete: Libraries imported and seeds set.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Provide the path to your dataset in Google Drive
data_path = '/content/drive/My Drive/Conversational_semantic_search/Customer_service.csv'
try:
    df = pd.read_csv(data_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file at {data_path} was not found.")

Dataset loaded successfully.


Loading and Reading the dataset

### **Preprocessing: Cleaning Text**

In [ ]:
def clean_text(text):
    text = str(text).lower().strip()
    text = text.replace("\n", " ").replace("\r", " ")
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

# Cleaning text
df['cleaned_instruction'] = df['instruction'].apply(clean_text)
df['cleaned_response'] = df['response'].apply(clean_text)

# Dropping rows with any missing values
df.dropna(subset=['cleaned_instruction', 'cleaned_response'], inplace=True)

df[['instruction', 'cleaned_instruction', 'response', 'cleaned_response']].head()


,instruction,cleaned_instruction,response,cleaned_response
0,question about cancelling order {{Order Number}},question about cancelling order order number,I've understood you have a question regarding ...,ive understood you have a question regarding c...
1,i have a question about cancelling oorder {{Or...,i have a question about cancelling oorder orde...,I've been informed that you have a question ab...,ive been informed that you have a question abo...
2,i need help cancelling puchase {{Order Number}},i need help cancelling puchase order number,I can sense that you're seeking assistance wit...,i can sense that youre seeking assistance with...
3,I need to cancel purchase {{Order Number}},i need to cancel purchase order number,I understood that you need assistance with can...,i understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase {{...",i cannot afford this order cancel purchase ord...,I'm sensitive to the fact that you're facing f...,im sensitive to the fact that youre facing fin...


In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
embeddings_instructions = []
embeddings_responses = []
# Smaller Batch-size results in accurate results
batch_size = 16

# Generator implementation for batch processing
def text_generator(df, column, batch_size):
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        yield df[column][start:end].tolist()

# Processing text in batches and saving embeddings
for i, batch in enumerate(text_generator(df, 'cleaned_instruction', batch_size)):
    batch_embeddings = model.encode(batch)
    embeddings_instructions.extend(batch_embeddings)
    # Save every 10 batches
    if i % 10 == 0:
        np.save('/content/drive/My Drive/Conversational_semantic_search/embeddings_instructions_partial.npy', np.array(embeddings_instructions))
        print(f"Processed {i * batch_size} instruction entries so far.")

for i, batch in enumerate(text_generator(df, 'cleaned_response', batch_size)):
    batch_embeddings = model.encode(batch)
    embeddings_responses.extend(batch_embeddings)
    # Save every 10 batches
    if i % 10 == 0:
        np.save('/content/drive/My Drive/Conversational_semantic_search/embeddings_responses_partial.npy', np.array(embeddings_responses))
        print(f"Processed {i * batch_size} response entries so far.")

# Final save
embeddings_instructions = np.array(embeddings_instructions)
embeddings_responses = np.array(embeddings_responses)
np.save('/content/drive/My Drive/Conversational_semantic_search/embeddings_instructions.npy', embeddings_instructions)
np.save('/content/drive/My Drive/Conversational_semantic_search/embeddings_responses.npy', embeddings_responses)
print("Embeddings instructions shape:", embeddings_instructions.shape)
print("Embeddings responses shape:", embeddings_responses.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed 0 instruction entries so far.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings instructions shape: (26872, 768)
Embeddings responses shape: (26872, 768)


OSError: Cannot save file into a non-existent directory: '/content/drive/My Drive'

In [ ]:
df[['instruction', 'cleaned_instruction', 'response', 'cleaned_response']].to_csv('/content/drive/My Drive/Conversational_semantic_search/Customer_service_preprocessed_.csv', index=False)
print("Done!")

Done!
